# Handout 02
#### Sara Díaz del Ser
_(in collaboration with Paula Romero)_

In [1]:
from tqdm.notebook import tqdm
import random
from collections import defaultdict, Counter
import timeit
import numpy as np
import re
from itertools import product
from copy import deepcopy

### Ex. 1 _(4 pts)_ One hundred ways to get to one hundred (more or less)

Write a program that outputs all possibilities to generate the number 100 from the digits
1, 2, 3, 4, 5, 6, 7, 8, 9 (in that order) either by putting a ‘+’, ‘−’, ‘∗’, ‘/’, or nothing between
the digits, e.g. 1 + 2 + 3 −4 + 5 + 6 + 78 + 9 = 100 or 1 + 23 ∗4 + 56/7 + 8 −9 = 100. 
#### Usual arithmetic rules apply. How many possibilities exist?

Hint: You can construct all possible expressions as strings and use the eval function.
You might also want to at least look at the itertools module of Python, specifically
the combinatoric generators to avoid deeply nested loops. Don’t forget: recursion is also
(always) a possibility. . .

In [2]:
result = 0
number = 100
for i in tqdm(product(['+','-','*','/',''],repeat=8)):
    s = '1'
    for operator,digit in zip(i,'23456789'):
        s = str(s + operator + digit)
        count = eval(s)
        if(count==number):
            result += 1
print("Number of possibilities to generate the number 100: ", result)

0it [00:00, ?it/s]

Number of possibilities to generate the number 100:  376


### Ex. 3 _(4 pts)_ Forbidden letters

#### (a) _(0 pts)_ Write a function named avoids that takes a word and a string of forbidden letters, and that returns True if the word doesn’t use any of the forbidden letters.
(See exercise 9.3 of ThinkPython).

In [3]:
def avoids(word:str, forbidden:str) -> bool:
    """Returns True if the input word doesn't use any of the forbidden letters."""
    regex = '|'.join(list(forbidden))
    if re.search('[{0}]'.format(regex), word):
        return False
    return True

In [4]:
# Example
print(f"Avoids zby? {avoids('supercalifragilisticexpialidocious', 'zby')}")
print(f"Avoids skj? {avoids('supercalifragilisticexpialidocious', 'skj')}")

Avoids zby? True
Avoids skj? False


#### (b) _(4 pts)_ Write some Python code to identify a combination of 6 forbidden letters that aims to exclude the smallest number of words from the file words.txt?
(Hint: You could start with the letter contained in the fewest words and go from there. Your
solution does not have to find the absolute best combination of 6 letters, but your
solution should be “reasonably” good.)

In [5]:
def forbidden_letter(file:str) -> str:
    """Finds the letter that appears in the smallest number of words in the input file"""
    with open (file, 'r') as f:
        # Create a flattened list of all the letters
        word_list = f.read().split()
        letters = [ list(word) for word in word_list ]
        flattened_list = [item for sublist in letters for item in sublist]
        # Count them
        count = Counter(flattened_list)
        # Order the list from most common (first) to least common (last)
        least_common = count.most_common()[-1]
        return least_common[0]

In [6]:
# Example
start = timeit.default_timer()
results = forbidden_letter(file='words.txt')
print(f"Forbidden letter: {results}")
stop = timeit.default_timer()
print('Time: ', stop - start)

Forbidden letter: q
Time:  0.28072309299999887


In [7]:
def forbidden_letters(file:str, n:int=6) -> str:
    """Finds the 6 letters that appears in the smallest number of words in the input file"""
    with open (file, 'r') as f:
        # Create a flattened list of all the letters
        word_list = f.read().split()
        letters = [ list(word) for word in word_list ]
        flattened_list = [item for sublist in letters for item in sublist]
        # Count them
        count = Counter(flattened_list)
        # Order the list from most common (first) to least common (last)
        least_common = count.most_common()[-n:]
        return [ each[0] for each in least_common ]

In [8]:
# Example
start = timeit.default_timer()
letters = forbidden_letters(file='words.txt')
print(f"Forbidden letters: {letters}")
stop = timeit.default_timer()
print('Time: ', stop - start)

Forbidden letters: ['v', 'w', 'z', 'x', 'j', 'q']
Time:  0.1810955659999962


In [9]:
# Check how many words are left if we remove the 6 forbidden letters
def words_remain(file:str, forbidden_letters:list) -> float:
    """Returns percentage of words left if we remove the 6 forbidden letters."""
    with open (file, 'r') as f:
        word_list = f.read().split()
        remain = [ word for word in word_list if avoids(word, forbidden="".join(forbidden_letters)) ]
        return (len(remain)/len(word_list)) * 100

In [10]:
# Example
start = timeit.default_timer()
results = words_remain(file='words.txt', forbidden_letters=letters)
print(f"Words remaining: {round(results, 2)}%")
stop = timeit.default_timer()
print('Time: ', stop - start)

Words remaining: 77.68%
Time:  0.23290224600000187


### Ex. 4 _(5 pts)_ A nice word puzzle and an even better programming exercise

#### What is the longest English word, that remains a valid English word, as you remove its letters one at a time?

Now, letters can be removed from either end, or the middle, but you can’t rearrange any
of the letters. Every time you drop a letter, you wind up with another English word. If
you do that, you’re eventually going to wind up with one letter and that too is going to be
an English word—one that’s found in the dictionary. We want to know what’s the longest
word and how many letters does it have?

Here is a little modest example: Sprite. You start off with sprite, you take a letter off, one
from the interior of the word, take the r away, and we’re left with the word spite, then we
take the e off the end, we’re left with spit, we take the s off, we’re left with pit, it, and I. (Ex.
12.4 from Think Python, adapted from http://www.cartalk.com/content/puzzlers.)

Write a program to find all words that can be reduced in this way, and then find the longest
one. Use the file words.txt as a dictionary of valid words for this exercise.

This exercise is a little more challenging than most and a few hints are given for solving this
problem are given in Think Python. When trying this exercise please note that the provided
list of words words.txt does not contain the one letter words ‘a’ and ‘i’, you might want
to add them to your list of words. (Depending on your approach you might also want to
add the empty word ' ' )

In [11]:
# Turn word list into a dictionary (add a and i)
def create_dictionary(file:str) -> dict:
    """Create a dictionary from a list of words in a .txt file"""
    with open (file, 'r') as f:
        # Create list of all the letters
        word_list = f.read().split()
        # Manually add 'a' and 'i'
        word_list.append('a')
        word_list.append('i')
        return { word:True for word in word_list }

In [12]:
def is_real(word:str, word_dic:dict) -> bool:
    """Checks if word exists in dictionary"""
    if word_dic.get(word):
        return True
    return False

In [13]:
def is_reducible(word:str, word_dic:dict) -> bool:
    """Checks if word can be reduced to other existing words"""        
    # Get all possible words if you remove one letter
    possible_words = [ word[:i] + word[i+1:] for i in range(len(word)) if word_dic.get(str(word[:i] + word[i+1:]))]

    for new_word in possible_words:
        # Check if the remaining word exists
        if is_real(new_word, word_dic):
            
            # Only continue to reduce it of it's more than 1 letter long
            if len(new_word) > 1:
                return is_reducible(new_word, word_dic)  
                
            elif len(new_word) == 1:
                return True     
            
        return False

In [14]:
def all_reducible(dic_file:str='words.txt'):
    """Find find all the words that can be reduced one letter a a timee and still be an existing word"""
    # Create word dic
    word_dic = create_dictionary(file=dic_file)

    # Check every word in the dic to see if it's reducible
    reducible_list = [ word for word in list(word_dic.keys()) if is_reducible(word=word, word_dic=word_dic) ] 

    return reducible_list

In [15]:
# Get all the reducible words from the list
start = timeit.default_timer()
reducible_list = all_reducible()
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  1.112044699000002


In [16]:
# Get the longest one
print(f"Found {len(reducible_list)} reducible words in the given dictionary.")
print(f"Longest reducible word in the given dictionary is: '{max(reducible_list, key = len)}'")

Found 7024 reducible words in the given dictionary.
Longest reducible word in the given dictionary is: 'complecting'


### Ex. 2 _(7 pts)_ Eight queens

The game of chess is played on a checkered board consisting of 64 squares arranged on an
8 by 8 grid. Different types of pieces are placed on the board, each of which is allowed
different types of moves. A piece can capture another piece (which is subsequently removed
from the board) if it can move to the square occupied by the other piece in a single turn.
Here, we are not really interested in the rules of the game, instead, we are only interested
in the movements of the piece called queen. The queen is the most powerful piece in chess,
it can move along the board in any direction, horizontally, vertically one or more squares.
Thus any other piece placed in a straight horizontal, vertical, or diagonal line (with no
intervening pieces) can be captured by the queen. The “eight queens” problem asks you to
place as many queens as possible on a chessboard so that no queen can capture any other
queen. (Note, that there cannot be more than 8 queens on an 8 by 8 chessboard, as this
requires at least one row and column to contain more than one queen, each of which would
be able to capture the other.)

#### (a) _(4 pts)_ Write a program that finds and prints a solution for the eight queens problem, placing eight queens on the board in such a way that no queen can capture any other. 

(In chess, columns are labeled a to h (left to right) and rows 1 to 8 (bottom to top). You can print the positions as a1, b5, c3, . . . for the eight queens.)

In [17]:
def is_available(pos) -> bool:
    """Check if a queen can be placed without being caught in the given position on the board"""
    # Check is spot is viable for a queen
    m,n = pos
    size = len(board)

    # Check out horizontal (rows) and horizontal (columns)
    for k in range(0,size):
        if board[m][k]==1 or board[k][n]==1:
            return False

    # Check out in diagonal
    for k in range(0,size):
        for l in range(0,size):
            if (k+l==m+n) or (k-l==m-n):
                if board[k][l]==1:
                    return False
    return True


In [18]:
def arrange_queens(n:int=8, start:int=0) -> bool:
    """Arrange a given number of queens on the board so they won't capture eachother"""
    if n==0:
        return True

    for j in range(0,N):

        # If the spot is available
        if (is_available((start,j))) and (board[start][j]!=1):

            # Add queen
            board[start][j] = 1

            # Backtrack
            if arrange_queens(n-1, start=start+1):
                return True
            

            board[start][j] = 0
    return False

In [19]:
# Create an 8x8 empty matrix
N = 4
board = np.zeros((N, N), dtype=int)
queens_placed = []

# Arrange n queens on the board
arrange_queens(N)
print(f'Board:\n{board}')

# Find where the queens are placed
for i in range(0,N):
    for j in range(0,N):
        if board[i][j]==1:

            # Add to list as letter-number combo:  
            # letter should be +1 (python's num start at 0), 
            # numbers go from bottom (1) to top (8)

            queens_placed.append(f"{-i+N}{chr(ord('`')+j+1)}")

# Print out queen coordinates
if len(queens_placed) == N:
    print(f'\nSuccessfully placed {N} queens: {queens_placed}')


Board:
[[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]

Successfully placed 4 queens: ['4b', '3d', '2a', '1c']


#### (b) _(0 pts)_ Make sure your program is able to also solve the more general problem of placing n queens on an n ×n board.

In [20]:
# Changed the 8 to 6 --  works! (anything over 10 will take foreveeeer to load)
# I tried using a wrapper function but since I'm not returning the board variable it doesn't work

N = 6
board = np.zeros((N, N), dtype=int)
queens_placed = []
# Arrange n queens on the board
arrange_queens(N)
print(f'Board:\n{board}')

# Find where the queens are placed
for i in range(0,N):
    for j in range(0,N):
        if board[i][j]==1:
            # Add to list as letter-number combo:  
            # letter should be +1 (python's num start at 0), 
            # numbers go from bottom (1) to top (8)
            queens_placed.append(f"{-i+N}{chr(ord('`')+j+1)}")

# Print out queen coordinates
if len(queens_placed) == N:
    print(f'\nSuccessfully placed {N} queens: {queens_placed}')

Board:
[[0 1 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 0 1]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]]

Successfully placed 6 queens: ['6b', '5d', '4f', '3a', '2c', '1e']


#### (c)  _(3 pts)_ Modify your program to find the total number of possible solutions to the n queens problem instead of a single solution. 

(You do not need to print all the solutions, just determine the number of possible solutions!) 

In [21]:
def arrange_queens(n:int=8, start:int=0) -> bool:
    """Arrange a given number of queens on the board so they won't capture eachother"""
    if n==0:
        global solutions
        
        # add current solution to total
        solutions.append(deepcopy(board))
        return True

    for j in range(0,N):
        # If the spot is available
        if (is_available((start,j))) and (board[start][j]!=1):

            # Add queen
            board[start][j] = 1

            # Backtrack
            arrange_queens(n-1, start=start+1)
            
            # Clear
            board[start][j] = 0
            
    return False

In [32]:
# Create an 8x8 empty matrix
N = 4
board = np.zeros((N, N), dtype=int)
solutions = []

# Arrange n queens on the board
arrange_queens(N)

print("Unique solutions:\n",len(solutions))

for i, sol in enumerate(solutions):
    print(f"\nSolution Board {i}:\n",sol)

Unique solutions:
 2

Solution Board 0:
 [[0 1 0 0]
 [0 0 0 1]
 [1 0 0 0]
 [0 0 1 0]]

Solution Board 1:
 [[0 0 1 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 1 0 0]]


#### What is the smallest n, for which at least one solution exists? What is the smallest n, for which more than one solution exists? 
(For this exercise you do not need to print the individual solutions.)


* The smallest n for which at least one solution exists is n=0 (since tecnically the board will be empty). The next one would be n=1, that also has 1 solution.
* The smallest n for which more than one solution exists is n=4, which has 2 possible solutions.

#### (d) _(0 pts)_ Depending on your approach and implementation, you might be able to also determine the number of possible solutions for the n queens problems also for larger n > 8 in a reasonable time. How large is n allowed to be so that your implementation/algorithm finds the number of possible solution within a few seconds?

* n=10 is the largest implementation of the algorithm that does not take more than a few seconds to solve